In [1]:
#!pip install datatable > /dev/null

import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns
import datatable as dt

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, roc_curve

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

import tidalUtl.PrpUtl as prp
import tidalUtl.EdaUtl as eda
from sklearn.model_selection import GroupKFold

In [2]:
print(torch.__version__) 

1.7.0+cu110


# Version

__ver1__<br>
baseline：CV:0.01465 LB:0.01874<br>


# Config

In [3]:
MODEL = "Pytorch"

In [4]:
INPUT = "/home/tidal/ML_Data/JaneStreet/jane-street-market-prediction"
OUTPUT = "/home/tidal/ML_Data/JaneStreet/output"
#INPUT = "/Users/hfuis/ML_Data/aneStreet/jane-street-market-prediction"
#OUTPUT = "/Users/hfuis/ML_Data/aneStreet/jane-street-market-prediction"

SUBMIT = OUTPUT + "/submittion/"
SAVEMODEL = OUTPUT + "/model/" + MODEL +"/"
SAVEOOF = OUTPUT + "/OOF/" + MODEL +"/"

In [5]:
%%time
#Loading
train_data_datatable = dt.fread(INPUT + '/train.csv')
trainFeature = train_data_datatable.to_pandas()

CPU times: user 38.9 s, sys: 1.28 s, total: 40.2 s
Wall time: 3.39 s


In [6]:
features = [c for c in trainFeature.columns if 'feature' in c]

In [7]:
#Seed固定
def seed_everything(seed=42):
    #data取得についてのランダム性固定
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #cudnnによる演算の安定化(評価値の安定)
    torch.backends.cudnn.deterministic = True
    
    #os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(seed=42)

In [8]:
#HyperParameter
param_space = {'hidden_size1': 512, 
               'hidden_size2': 512, 
               'dropOutRate1': 0.20393004966355735, 
               'dropOutRate2': 0.39170486751620137,
               'rankGauss_n_quantiles': 488.0393350201078,
               'leakyReluSlope': 0.01973893854348531,
              }

# Preprocessing

In [9]:
trainFeature

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1,-1.872746,-2.191242,...,NaN,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807,0
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,...,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1
2,0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1,0.812780,-0.256156,...,NaN,6.115747,9.667908,5.542871,11.671595,7.281757,10.060014,6.638248,9.427299,2
3,0,0.000000,-0.004730,-0.003273,-0.000461,-0.000476,-0.003200,-1,1.174378,0.344640,...,NaN,2.838853,0.499251,3.033732,1.513488,4.397532,1.266037,3.856384,1.013469,3
4,0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1,-3.172026,-3.093182,...,NaN,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390486,499,0.000000,0.000142,0.000142,0.005829,0.020342,0.015396,1,-1.649365,-1.169996,...,-1.896874,-1.260055,1.947725,-1.994399,-1.685163,-2.866165,-0.216130,-1.892048,0.901585,2390486
2390487,499,0.000000,0.000012,0.000012,-0.000935,-0.006326,-0.004718,1,2.432943,5.284504,...,-0.936553,1.064936,3.119762,-0.419796,-0.208975,-0.146749,0.730166,0.648452,2.068737,2390487
2390488,499,0.000000,0.000499,0.000499,0.007605,0.024907,0.016591,1,-0.622475,-0.963682,...,-2.956745,-0.640334,-2.279663,-0.950259,-4.388417,-1.669922,-3.288939,-1.336142,-2.814239,2390488
2390489,499,0.283405,-0.000156,-0.000156,-0.001375,-0.003702,-0.002004,-1,-1.463757,-1.107228,...,-2.035894,-1.780962,0.881246,-2.202140,-1.912601,-3.341684,-0.571188,-2.185795,0.627452,2390489


In [10]:
features

['feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_19',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_24',
 'feature_25',
 'feature_26',
 'feature_27',
 'feature_28',
 'feature_29',
 'feature_30',
 'feature_31',
 'feature_32',
 'feature_33',
 'feature_34',
 'feature_35',
 'feature_36',
 'feature_37',
 'feature_38',
 'feature_39',
 'feature_40',
 'feature_41',
 'feature_42',
 'feature_43',
 'feature_44',
 'feature_45',
 'feature_46',
 'feature_47',
 'feature_48',
 'feature_49',
 'feature_50',
 'feature_51',
 'feature_52',
 'feature_53',
 'feature_54',
 'feature_55',
 'feature_56',
 'feature_57',
 'feature_58',
 'feature_59',
 'feature_60',
 'feature_61',
 'feature_62',
 'feature_63',
 'feature_64',
 'feature_65',
 'feature_66',
 'fea

## Func: In & Out Type is DataFrame

### Fill NAN

In [11]:
def FillNAN(trainFeature_inp):
    trainFeature = trainFeature_inp.copy()
    
    f_mean = trainFeature[features[1:]].mean()
    trainFeature = trainFeature.query('weight > 0').reset_index(drop = True)
    
    #欠損値を平均で埋める
    trainFeature[features[1:]] = trainFeature[features[1:]].fillna(f_mean)
    
    #targetを生成
    trainFeature['action'] = (trainFeature['resp'] > 0).astype('int')
    
    
    
    return trainFeature

## Collecting

__train,testにターゲット値も連結__

In [12]:
def Collecting(trainFeature_inp):
    trainFeature = trainFeature_inp.copy()
    
    #targetを生成
    trainFeature['action'] = (trainFeature['resp'] > 0).astype('int')
    
    #target単体でも保持。
    target = trainFeature['action'].copy()
    target = pd.DataFrame(target,columns=["action"])
    return trainFeature, target

## Preprocessing Summary

In [13]:
def preprocessing(param, trainFeature):
    
    #欠損値処理
    trainFeature = FillNAN(trainFeature)
    
    #trainにターゲット値を連結。
    train, target = Collecting(trainFeature)
    
    
    return train, target

## Work

## Visualization

In [14]:
%%time
trainVsl, targetVsl = preprocessing(param_space, trainFeature)

CPU times: user 3.19 s, sys: 1.03 s, total: 4.21 s
Wall time: 4.21 s


In [15]:
trainVsl.head(5)

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id,action
0,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,...,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1,0
1,0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1,-3.172026,-3.093182,...,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633,4,0
2,0,0.190575,-0.001939,-0.002301,0.001088,0.005963,0.000709,-1,-3.172026,-3.093182,...,0.336873,4.076447,0.614783,6.622176,0.800618,5.231595,0.361506,3.921714,6,1
3,0,3.820844,0.017395,0.021361,0.031163,0.036970,0.033473,-1,0.446050,-0.466210,...,2.101997,4.846202,1.479875,5.261328,2.305066,4.571762,2.201537,4.429745,7,1
4,0,0.116557,-0.005460,-0.007301,-0.009085,-0.003546,-0.001677,1,-3.172026,-3.093182,...,1.537913,4.785838,1.637435,6.968002,2.354338,5.825499,1.778029,4.740577,8,0


In [16]:
targetVsl.head(5)

,action
0,0
1,0
2,1
3,1
4,0


In [17]:
type(targetVsl)

pandas.core.frame.DataFrame

In [18]:
print("Train: "+ str(trainVsl.shape))
print("Target: "+ str(targetVsl.shape))

Train: (1981287, 139)
Target: (1981287, 1)


# Fitting

## Config about Fitting

In [19]:
#configは辞書化しておく。
def Config_about_Fitting(train, target, folds):
    confFitting = {}
    
    #Fitするときに"y"として使う列の列名配列
    confFitting["target_cols"] = target.columns.values.tolist()
    #Fitするときに"X"として使う列の列名配列
    #kfold, id等はここで削除。
    feature_cols = [c for c in folds.columns if c not in confFitting["target_cols"]]
    confFitting["feature_cols"] = [c for c in feature_cols if c not in ['kfold','sig_id']]
    #特徴量、ターゲットのサイズ
    confFitting["num_features"]=len(confFitting["feature_cols"])
    confFitting["num_targets"]=len(confFitting["target_cols"])
    
    return confFitting

## Dataset Classes

In [20]:
#Train,Valid用のデータクラス
class TrainDataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        #torch.DataLoaderに入れるための形式
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
#Test用のデータクラス
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            #torch.DataLoaderに入れるための形式
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

## Loss, Metric

In [21]:
#loss
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self):
        super(LabelSmoothingCrossEntropy, self).__init__()
    def forward(self, x, target, smoothing=0.001):
        confidence = 1. - smoothing
        logprobs = F.log_softmax(x, dim=-1)
        bcs_loss = nn.BCEWithLogitsLoss()(x, target)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * bcs_loss + smoothing * smooth_loss
        return loss.mean()

In [22]:
#metric
#roc_auc_score(y_val, oof[te])

## Func: Fitting, Evaluation, Predict

In [23]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(targets, outputs)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   
    

## Model architect

In [24]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets, param):
        super(Model, self).__init__()
        #hyperoptによる被探索パラメータ
        hidden_size1=param['hidden_size1']
        hidden_size2=param['hidden_size2']
        dropOutRate1=param['dropOutRate1']
        dropOutRate2=param['dropOutRate2']
        leakyReluSlope=param['leakyReluSlope']
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(dropOutRate1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size1))
        self.leakyRelu1 = nn.LeakyReLU(negative_slope=leakyReluSlope)
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size1)
        self.dropout2 = nn.Dropout(dropOutRate2)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size1, hidden_size2))
        self.leakyRelu2 = nn.LeakyReLU(negative_slope=leakyReluSlope)
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size2)
        self.dropout3 = nn.Dropout(dropOutRate2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size2, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = self.leakyRelu1(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = self.leakyRelu2(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

# Run

## HyperParameter

In [25]:
# HyperParameters
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 512
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = True

## CV folds

In [26]:
def CV_folds(train, target):
    folds = train.copy()
    
    gkf = GroupKFold(n_splits=NFOLDS)
    
    for f, (t_idx, v_idx) in enumerate(gkf.split(train, target, train["date"])):
        folds.loc[v_idx, 'kfold'] = int(f)
    
    folds['kfold'] = folds['kfold'].astype(int)
    
    return folds

In [27]:
%%time
#Preprocessing Data
trainVsl, targetVsl = preprocessing(param_space, trainFeature)
#CV folds
foldsVsl = CV_folds(trainVsl, targetVsl)

foldsVsl.head(5)

CPU times: user 4.06 s, sys: 1.7 s, total: 5.76 s
Wall time: 5.77 s


,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id,action,kfold
0,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,...,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1,0,4
1,0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1,-3.172026,-3.093182,...,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633,4,0,4
2,0,0.190575,-0.001939,-0.002301,0.001088,0.005963,0.000709,-1,-3.172026,-3.093182,...,4.076447,0.614783,6.622176,0.800618,5.231595,0.361506,3.921714,6,1,4
3,0,3.820844,0.017395,0.021361,0.031163,0.036970,0.033473,-1,0.446050,-0.466210,...,4.846202,1.479875,5.261328,2.305066,4.571762,2.201537,4.429745,7,1,4
4,0,0.116557,-0.005460,-0.007301,-0.009085,-0.003546,-0.001677,1,-3.172026,-3.093182,...,4.785838,1.637435,6.968002,2.354338,5.825499,1.778029,4.740577,8,0,4


In [28]:
foldsVsl['kfold'].value_counts()

3    397380
0    396305
2    396003
1    395800
4    395799
Name: kfold, dtype: int64

## Single Fold Running

In [29]:
def run_training(confFitting, Tester, fold, seed, param,
                 folds, train, target):
    
    seed_everything(seed)
    
    train = folds
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[confFitting["feature_cols"]].values, train_df[confFitting["target_cols"]].values
    x_valid, y_valid =  valid_df[confFitting["feature_cols"]].values, valid_df[confFitting["target_cols"]].values
    
    train_dataset = TrainDataset(x_train, y_train)
    valid_dataset = TrainDataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=confFitting["num_features"],
        num_targets=confFitting["num_targets"],
        param=param
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    ##### 評価関数 ######
    train_loss_fn = LabelSmoothingCrossEntropy()
    valid_loss_fn = roc_auc_score()
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, train_loss_fn, trainloader, DEVICE)
        valid_loss, valid_preds = valid_fn(model, valid_loss_fn, validloader, DEVICE)
        if Tester:
            print("EPOCH: {:03}: | train_loss: {:.3f}: | valid_loss: {:.3f}".format(epoch, train_loss, valid_loss))
        
        if valid_loss > best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"{SAVEMODEL}SEED{seed}_FOLD{fold}.pth")
        
        elif(EARLY_STOP == True):
            early_step += 1
            if (early_step >= early_stopping_steps):
                if Tester:
                    print('Early stopping. Best Val loss: {:.3f}'.format(best_loss))
                break
            
    
    ################本コンペの仕様上テストデータがない#############################################
    
    return oof
    
    #--------------------- PREDICTION---------------------
    #x_test = test[confFitting["feature_cols"]].values
    #testdataset = TestDataset(x_test)
    #testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    #model = Model(
    #    num_features=confFitting["num_features"],
    #    num_targets=confFitting["num_targets"],
    #    param=param
    #)
    #
    #model.load_state_dict(torch.load(f"{SAVEMODEL}SEED{seed}_FOLD{fold}.pth"))
    #model.to(DEVICE)
    #
    #predictions = np.zeros((len(test), target.iloc[:, 1:].shape[1]))
    #predictions = inference_fn(model, testloader, DEVICE)
    #
    #
    #return oof, predictions


## K-Fold Running

In [30]:
def run_k_fold(Tester, NFOLDS, seed, param,
              folds, train, target, confFitting):
    oof = np.zeros((len(train), confFitting["num_targets"]))
    #predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    for fold in range(NFOLDS):
        if Tester:
            print('=' * 20, 'Fold', fold, '=' * 20)
        oof_= run_training(confFitting, Tester, fold, seed, param,
                                   folds, train, target)
        
        #predictions += pred_ / NFOLDS
        oof += oof_
        
    #return oof, predictions
    return oof

## CV Evaluation

In [31]:
 def CV_Evaluation(confFitting, oof, train, target):
    #CV score : OOFの評価結果。
    #OOF(学習モデルによるtrain dataの予測)
    train[confFitting["target_cols"]] = oof
    #target(予測結果)：ここで処理「cp_type = ctl_vehicleのレコードを削除」で抜けたところに0を入れている。
    valid_results = trainTargetScored.drop(columns=confFitting["target_cols"]).merge(train[['sig_id']+confFitting["target_cols"]], on='sig_id', how='left').fillna(0)
    
    y_true = trainTargetScored[confFitting["target_cols"]].values
    y_pred = valid_results[confFitting["target_cols"]].values
    
    score = 0
    for i in range(confFitting["num_targets"]):
        score_ = log_loss(y_true[:, i], y_pred[:, i]) #問題の評価指標によって変わる。
        score += score_ / target.shape[1]
        
    print("CV log_loss: ", score)
    
    #OOF save
    np.save(SAVEOOF + 'oof', y_pred)
    
    return score

## Postprocessing

In [32]:
# 特になし

## Submit

In [33]:
def Submit(confFitting, predictions, test):
    test[confFitting["target_cols"]] = predictions
    sub = sample_submission.drop(columns=confFitting["target_cols"]).merge(test[['sig_id']+confFitting["target_cols"]], on='sig_id', how='left').fillna(0)
    sub.to_csv(f'{SUBMIT}submission.csv', index=False)

    print("sub.shape" + str(sub.shape))
    
    return

# Execute

In [34]:
def Exec(param):
    
    #Tester(True/False)
    Tester = True
    
    #Preprocessing Data
    train, target = preprocessing(param_space, trainFeature)
    
    #CV folds
    folds = CV_folds(train, target)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, target, folds)
    
    # Averaging on multiple SEEDS
    SEED = [0, 1, 2, 3 ,4, 5]
    oof = np.zeros((len(train), confFitting["num_targets"]))
    #predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        oof_ = run_k_fold(Tester, NFOLDS, seed, param,
                                       folds, train, target, confFitting)
        oof += oof_ / len(SEED)
        #predictions += predictions_ / len(SEED)
    
    #CV 評価
    score = CV_Evaluation(confFitting, oof, train, target)
    
    # 課題提出
    #Submit(confFitting, predictions, test)
    
    return score


In [35]:
%%time
score= Exec(param_space)
print("score: " + str(score))

~~~~~~~~~~~~~~~~~~~~ SEED 0 ~~~~~~~~~~~~~~~~~~~~
==================== Fold 0 ====================


TypeError: roc_auc_score() missing 2 required positional arguments: 'y_true' and 'y_score'

# Predict

In [7]:
def run_predict(confFitting, param, test, target, fold, seed):
    
    seed_everything(seed)
  
    #--------------------- PREDICTION---------------------
    x_test = test[confFitting["feature_cols"]].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=confFitting["num_features"],
        num_targets=confFitting["num_targets"],
        param=param
    )
    
    model.load_state_dict(torch.load(f"{SAVEMODEL}SEED{seed}_FOLD{fold}.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    
    return predictions


In [8]:
def run_k_fold_predict(confFitting, test, target, param, Tester, NFOLDS, seed):
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    for fold in range(NFOLDS):
        if Tester:
            print('=' * 20, 'Fold', fold, '=' * 20)
        pred_ = run_predict(confFitting, param, test, target, fold, seed)
        
        predictions += pred_ / NFOLDS
        
    return predictions

In [45]:
def SubmitPredict(confFitting, predictions, test, prefix):
    test[confFitting["target_cols"]] = predictions
    sub = sample_submission.drop(columns=confFitting["target_cols"]).merge(test[['sig_id']+confFitting["target_cols"]], on='sig_id', how='left').fillna(0)
    sub.to_csv(f'{SUBMIT}{prefix}submission.csv', index=False)

    print("sub.shape" + str(sub.shape))
    
    return

In [46]:
def Predict(param):
    #Tester(True/False)
    Tester = False
    
    #Preprocessing Data
    train, target = preprocessing(param_space, trainFeature)
    
    #CV folds
    folds = CV_folds(train, target)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, test, target, folds)
    
    # Averaging on multiple SEEDS
    SEED = [0, 1, 2, 3 ,4, 5]
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        predictions_ = run_k_fold_predict(confFitting, test, target, param, Tester, NFOLDS, seed)
        predictions += predictions_ / len(SEED)
    
    # 課題提出
    prefix = "Pytorch"
    SubmitPredict(confFitting, predictions, test, prefix)
    
    return

In [65]:
%%time
Predict(param_space)

sub.shape(3982, 207)
CPU times: user 18.5 s, sys: 12.8 s, total: 31.3 s
Wall time: 6.22 s


# Hyperparameter Tuning

In [29]:
#hyperopt
from hyperopt import fmin, tpe, hp, rand, Trials

In [30]:
def HOptExec(param):
    #Tester(True/False)
    Tester = False
    
    #Preprocessing Data
    train, test, target = preprocessing(param, trainFeature, testFeature, trainTargetScored)
    
    #CV folds
    folds = CV_folds(train, target)
    
    #Config about Fitting
    confFitting = Config_about_Fitting(train, test, target, folds)
    
    # Averaging on multiple SEEDS
    SEED = [0, 1, 2, 3 ,4, 5]
    oof = np.zeros((len(train), confFitting["num_targets"]))
    predictions = np.zeros((len(test), confFitting["num_targets"]))
    
    ### RUN ###
    for seed in SEED:
        if Tester:
            print('~' * 20, 'SEED', seed, '~' * 20)
        oof_, predictions_ = run_k_fold(Tester, NFOLDS, seed, param,
                                       folds, train, test, target, confFitting)
        oof += oof_ / len(SEED)
        predictions += predictions_ / len(SEED)
    
    #CV 評価
    score = CV_Evaluation(confFitting, oof, train, target)
    
    # 課題提出
    #Submit(confFitting, predictions, test)
    
    return score

In [ ]:
%%time

param_space = {'hidden_size1': 512, 
               'hidden_size2': 512, 
               'dropOutRate1': 0.20393004966355735, 
               'dropOutRate2': 0.39170486751620137,
               'rankGauss_n_quantiles': 488.0393350201078,
               'leakyReluSlope': hp.uniform('leakyReluSlope', 1e-3, 1e-1),
              }

trials = Trials()

hopt = fmin(fn = HOptExec, 
            space = param_space, 
            algo = tpe.suggest, 
            max_evals = 15, 
            #timeout = 8.9 * 60 * 60, 
            trials = trials, 
           )

print(hopt)

CV log_loss:                                          
0.014981391207012364                                  
CV log_loss:                                                                         
0.01504250432043703                                                                  
CV log_loss:                                                                         
0.015004835293169368                                                                 
CV log_loss:                                                                         
0.015002514832957038                                                                 
CV log_loss:                                                                         
0.015008986227264749                                                                 
CV log_loss:                                                                         
0.014993115273980633                                                                   
CV log_loss:                